# Spam - Decision Tree

In [ ]:
emails=read.csv("../Csv_Files/emails.csv",stringsAsFactors=FALSE)

In [ ]:
str(emails)

In [ ]:
table(emails$spam)

In [ ]:
library(tm)

In [ ]:
library(SnowballC)

corpus - structure the collection of documents to make preprocessing easier

In [ ]:
corpus = Corpus(VectorSource(emails$text))

In [ ]:
corpus

Convert upper case to lower case

In [ ]:
corpus = tm_map(corpus, content_transformer(tolower))

Remove punctuations

In [ ]:
corpus = tm_map(corpus, removePunctuation)

Remove stopwords such as is, the etc

In [ ]:
corpus = tm_map(corpus, removeWords, stopwords("english"))

Stem the document. Replace removed, remove to remov

In [ ]:
corpus = tm_map(corpus, stemDocument)

create document frequency matrix

In [ ]:
dfm = DocumentTermMatrix(corpus)

In [ ]:
dfm

Extract only top frequenct words

In [ ]:
sdfm = removeSparseTerms(dfm, 0.995)

Convert the data to required dataframe format

In [ ]:
emailSparse = as.data.frame(as.matrix(sdfm))

In [ ]:
colnames(emailSparse) = make.names(colnames(emailSparse))

Find the word with maximun count

In [ ]:
which.max(colSums(emailSparse))

In [ ]:
emailSparse$spam = emails$spam

Find the word with large count in spam 

In [ ]:
colnames(emailSparse[colSums(subset(emailSparse,spam==1))>1000])

Find the word with large count in non spam 

In [ ]:
colnames(emailSparse[colSums(subset(emailSparse,spam==0))>=5000])

In [ ]:
colnames(emailSparse[colSums(subset(emailSparse,spam==1))>=1000])

In [ ]:
library(caTools)

In [ ]:
emailSparse$spam = as.factor(emailSparse$spam)

In [ ]:
table(emailSparse$spam)

In [ ]:
set.seed(123)

In [ ]:
spl = sample.split(emailSparse$spam, SplitRatio = 0.7)

In [ ]:
trainSparse = subset(emailSparse, spl==TRUE)

In [ ]:
table(trainSparse$spam)

In [ ]:
testSparse = subset(emailSparse, spl==FALSE)

In [ ]:
library(rpart)

In [ ]:
library(rpart.plot)

Build the decision tree model

In [ ]:
spamCART = rpart(spam ~ ., data=trainSparse, method="class",cp=0.1)

Predict for test case

In [ ]:
predictCART = predict(spamCART, newdata=testSparse, type="class")

In [ ]:
table(predictCART)

In [ ]:
table(testSparse$spam, predictCART)

In [ ]:
library(caret)

Set cross-validation as 10

In [ ]:
numFolds = trainControl( method = "cv", number = 10 )

In [ ]:
cpGrid = expand.grid( .cp = seq(0.01,0.5,0.01)) 

In [ ]:
train(spam ~.,data = trainSparse, method = "rpart", trControl = numFolds, tuneGrid = cpGrid )

In [ ]:
spamCART2 = rpart(spam ~ ., data=trainSparse, method="class",cp=0.01)

In [ ]:
predictCART = predict(spamCART2, newdata=testSparse, type="class")

In [ ]:
table(testSparse$spam, predictCART)